<a href="https://colab.research.google.com/github/kdmalc/fl-gestures/blob/main/ELEC_COMP_447_546_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELEC/COMP 447/546 Assignment 3


## General instructions
Please copy this colab notebook into your own Drive to edit. This notebook will also serve as your final submission report - please ensure that code cells run correctly, and that all non-code (text/latex) blocks are rendered correctly before submissing the file. Feel free to add any additional cells (code or text) you need. Please follow good coding, markdown, and presentation etiquette.

__Please do not use any AI tools for this assignment.__


## Submission instructions

- Before submitting, please `run-all` the code. This will re-render your entire jupyter file cell by cell to produce all the outputs.

- You are required to download the colab notebook as a `.ipynb` file and submit it to canvas. Please name your `.ipynb` file as `netid.ipynb`

- Modify the text cell on top to include your name and the names of any collaborators from this class you worked with on this assignment.

- Download a pdf of the executed colab notebook. You can use print -> save as pdf. Please name your `.pdf` file as `netid.pdf`.

- Any extra images used in the homework should also be uploaded to canvas.

- For simplicity, you can also upload a `netid.zip` file to canvas containing all solution files.

In [ ]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1.0 Implement the Lucas-Kanade algorithm

In this problem, you will implement the Lucas-Kanade algorithm and demonstrate tracking points
on a video

In [ ]:
"""
Some helper functions that are optional to use.

"""

def read_video_into_numpy(filename):
  cap = cv2.VideoCapture(filename)
  frames = []
  while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
      print("Can't receive frame (stream end?). Exiting ...")
      break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

  cap.release()
  video = np.stack(frames, axis=0)#converts to numpy array(T,H,W,C)
  video = np.transpose(video, (1,2,3,0))#(T,H,W,C)->(H,W,C,T)
  return frames

In [ ]:
def detect_corners(img):
  dst = cv2.cornerHarris(img, 2, 3, 0.04)
  y,x = np.where(dst > 0.1*dst.max()) # tuple of locations
  return x, y

In [ ]:
### Your code starts here ###

# Read the video into frames
frames = read_video_into_numpy('your video.mp4')


# Implement Lucas-Kanade




In [ ]:
# Save the gif
import imageio
imageio.mimsave('tracking.gif', im_list, fps=10)  # im_list is a list of your output images

## 3.0 Image Compression with PCA

In this problem, you will use PCA to compress images, by encoding small patches in low-
dimensional subspaces.

### 3.1 Use PCA to model patches

In [ ]:
"""
Some helper functions that are optional to use.

"""

# Sample patches from the image
def get_patches(img, patch_size=16, n_patches=1000, overlap=True):
  h, w, c = img.shape
  patches = np.zeros((n_patches, patch_size * patch_size * 3))
  for i in range(n_patches):
    x = np.random.randint(0, w - patch_size)
    y = np.random.randint(0, h - patch_size)

    patch = img[y:y+patch_size, x:x+patch_size, :].reshape((1, -1))

    patches[i, :] = patch
  if not overlap:
    patches = []
    for i in range(0, h, patch_size):
      for j in range(0, w, patch_size):
        if i<=h-patch_size and j<w-patch_size:
          patch = img[i:i+patch_size, j:j+patch_size, :].reshape((-1,))
          patches.append(patch)

    patches = np.asarray(patches)

  return patches

In [ ]:
### Your code starts here ###

# Load test image


# Get patches from image


# Implement PCA on the patches



# Display the first 36 principal components


### 3.2 Compress the image

In [ ]:
### Your code starts here ###

# Image reconstruction

